# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
  print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)





{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

### Question 1: What is the sum of the outputs of the generator for limit = 5?

In [4]:
import dlt


Q1_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')



info = Q1_pipeline.run(square_root_generator(limit),
										table_name="sqrt_value",
										write_disposition="replace")


print(info)



Pipeline dlt_colab_kernel_launcher load step completed in 0.41 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707923764.5566747 is LOADED and contains no failed jobs


In [5]:
import duckdb

conn = duckdb.connect(f"{Q1_pipeline.pipeline_name}.duckdb")


conn.sql(f"SET search_path = '{Q1_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))



print("\n\n\n sqrt_value table below:")

view = conn.sql("SELECT * FROM sqrt_value").df()
display(view)

sum = conn.sql("SELECT SUM(value) FROM sqrt_value").df()
display(sum)



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ sqrt_value          │
└─────────────────────┘




 sqrt_value table below:


,value,_dlt_load_id,_dlt_id
0,1.000000,1707923764.5566747,PfiVumfSdumEyA
1,1.414214,1707923764.5566747,u3sjAHljfaaKTg
2,1.732051,1707923764.5566747,sCvJcMgtmHMoDA
3,2.000000,1707923764.5566747,t3uxiNLML1afwQ
4,2.236068,1707923764.5566747,PtZE/PBHQAL+SQ


,"sum(""value"")"
0,8.382332


###Question 2: What is the 13th number yielded by the generator?

In [6]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
  print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


###Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

In [7]:
import dlt


Q3_pipeline = dlt.pipeline(destination='duckdb', dataset_name='persons')



info_1 = Q3_pipeline.run(people_1(),
										table_name="persons_1",
										write_disposition="replace")


print(info_1)

Pipeline dlt_colab_kernel_launcher load step completed in 0.51 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707923773.888841 is LOADED and contains no failed jobs


In [8]:
import duckdb

conn = duckdb.connect(f"{Q3_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{Q3_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))



print("\n\n\n persons_1 table below:")

P_key = conn.sql("SELECT * FROM persons_1").df()
display(P_key)

P_1 = conn.sql("SELECT SUM(age) FROM persons_1").df()
display(P_1)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons_1           │
└─────────────────────┘




 persons_1 table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707923773.888841,CfKlCOXeKLBrtA
1,2,Person_2,27,City_A,1707923773.888841,K/BCkkwFJM2WXA
2,3,Person_3,28,City_A,1707923773.888841,of4iZOFn6zSChg
3,4,Person_4,29,City_A,1707923773.888841,UJN3Khx1fzgZKQ
4,5,Person_5,30,City_A,1707923773.888841,GAUus1uoWlSdeg


,sum(age)
0,140.0


In [9]:

Q3p2_pipeline = dlt.pipeline(destination='duckdb', dataset_name='persons')


info_2 = Q3p2_pipeline.run(people_2(),
										table_name="persons_1",
										write_disposition="append",
                    primary_key="id")

print(info_2)

Pipeline dlt_colab_kernel_launcher load step completed in 0.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707923780.968784 is LOADED and contains no failed jobs


In [10]:

import duckdb

conn = duckdb.connect(f"{Q3_pipeline.pipeline_name}.duckdb")


conn.sql(f"SET search_path = '{Q3_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n persons_1 table below:")

P_all = conn.sql("SELECT * FROM persons_1").df()
display(P_all)

P_2 = conn.sql("SELECT SUM(age) FROM persons_1").df()
display(P_2)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons_1           │
└─────────────────────┘




 persons_1 table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707923773.888841,CfKlCOXeKLBrtA,None
1,2,Person_2,27,City_A,1707923773.888841,K/BCkkwFJM2WXA,None
2,3,Person_3,28,City_A,1707923773.888841,of4iZOFn6zSChg,None
3,4,Person_4,29,City_A,1707923773.888841,UJN3Khx1fzgZKQ,None
4,5,Person_5,30,City_A,1707923773.888841,GAUus1uoWlSdeg,None
5,3,Person_3,33,City_B,1707923780.968784,P6akaLPdrMBmAA,Job_3
6,4,Person_4,34,City_B,1707923780.968784,NfGjZRboMgygMg,Job_4
7,5,Person_5,35,City_B,1707923780.968784,f7eIyTLb433glQ,Job_5
8,6,Person_6,36,City_B,1707923780.968784,XtNi0/k7ZZ4YmQ,Job_6
9,7,Person_7,37,City_B,1707923780.968784,JWCy5FNAX6CDCw,Job_7


,sum(age)
0,353.0


###Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

In [11]:
import dlt


Q4_pipeline = dlt.pipeline(destination='duckdb', dataset_name='persons_1')



info_merge = Q4_pipeline.run(people_1(),
										table_name="persons_merge",
										write_disposition="replace")


print(info_merge)

Pipeline dlt_colab_kernel_launcher load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_1
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707923785.4857714 is LOADED and contains no failed jobs


In [12]:

import duckdb

conn = duckdb.connect(f"{Q4_pipeline.pipeline_name}.duckdb")


conn.sql(f"SET search_path = '{Q4_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))



print("\n\n\n persons_merge table below:")

P_key = conn.sql("SELECT * FROM persons_merge").df()
display(P_key)



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons_merge       │
└─────────────────────┘




 persons_merge table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707923785.4857714,WvYGbwXxxke5mA
1,2,Person_2,27,City_A,1707923785.4857714,lYqg3c88J8pHKg
2,3,Person_3,28,City_A,1707923785.4857714,tS+ByAAidCJaMg
3,4,Person_4,29,City_A,1707923785.4857714,N7FYjleYXAvZ/w
4,5,Person_5,30,City_A,1707923785.4857714,L4A5I12iQOQzZA


In [13]:

Q4p2_pipeline = dlt.pipeline(destination='duckdb', dataset_name='persons_1')


info_2_merge = Q4p2_pipeline.run(people_2(),
										table_name="persons_merge",
										write_disposition="merge",
                    primary_key="id")

print(info_2_merge)

Pipeline dlt_colab_kernel_launcher load step completed in 0.61 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_1
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707923792.5294826 is LOADED and contains no failed jobs


In [14]:

import duckdb

conn = duckdb.connect(f"{Q4p2_pipeline.pipeline_name}.duckdb")


conn.sql(f"SET search_path = '{Q4p2_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n persons_merge table below:")

P_all = conn.sql("SELECT * FROM persons_merge").df()
display(P_all)

P_2 = conn.sql("SELECT SUM(age) FROM persons_merge").df()
display(P_2)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons_merge       │
└─────────────────────┘




 persons_merge table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707923785.4857714,WvYGbwXxxke5mA,None
1,2,Person_2,27,City_A,1707923785.4857714,lYqg3c88J8pHKg,None
2,5,Person_5,35,City_B,1707923792.5294826,clYqf5GpVFF+Sw,Job_5
3,8,Person_8,38,City_B,1707923792.5294826,Yr7DUSryza9hfg,Job_8
4,7,Person_7,37,City_B,1707923792.5294826,vHXn62H9z4eCpA,Job_7
5,4,Person_4,34,City_B,1707923792.5294826,uyMKOWnkk8vk8g,Job_4
6,3,Person_3,33,City_B,1707923792.5294826,XDJ3i59oV/Y00g,Job_3
7,6,Person_6,36,City_B,1707923792.5294826,K9ppIgwZQU956g,Job_6


,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX